## Load relevant libraries

In [1]:
from datetime import date
import pandas as pd
import csv, os, math
from IPython.display import display
import ipywidgets as widgets


## Functions that need to be run

In [53]:
## template builder if there it is a new game starting that creates an empty file

def build_template(filename):
# STRING -> NONE
# file name -> outputs a file in the file name
    with open(filename, 'w',newline= '') as csvfile:
        rwwriter = csv.writer(csvfile, delimiter = ',')
        rwwriter.writerow(['Round', 'PandemicNum','Pandemic','City','Added'])
    csvfile.close()

In [29]:
## updates the lists with the things that have happened from a previous file

def catchup(city, trial_df):
    # LIST, DF -> LIST, LIST, LIST
    # list of cities, trial data frame, -> revealed deck, known deck, remains of starting deck
    
    # empty decks which will be filled later
    revealed = [];
    known = [];

    for index, row in trial_df.iterrows():
        if row.Pandemic == 1: # if its a pandemics add the 'bottom card' and remove the ones from known and put them into the revealed pile
            revealed.append(row.City)
            city.pop(city.index(row.City))
            known.insert(0,revealed)
            revealed = [];
            
        ## if the known deck and reveal deck are empty pull from the cities list and put them in the revealed deck
        elif not known:
            revealed.append(row.City)
            city.pop(city.index(row.City))
            
        # there are known cards in the known deck pull the the first known list
        elif known:
            revealed.append(row.City)
            known[0].pop(known[0].index(row.City))
            if not known[0]: # remove the first list if its empty
                known.pop(0)
    pandemicNum = trial_df.PandemicNum[len(trial_df)-1]
    roundNum = trial_df.Round[len(trial_df)-1]
    return(revealed, known, city, pandemicNum,roundNum)
    

In [4]:
## calcprobs calculates the probability of different cards based on their placement in the deck

def calcprobs(known, city, numOfPulls):
    # LIST, LIST, INT -> DF
    # List of list of known cities, list of all unrevealed cities, number of pulled cards

    thecities = [];
    probs = [];

    ## look through the known lists and calculate the probablity of it occuring within a certain number of turns 
    for i in range(len(known)):

        # loop through the subdeck
        for j in range(len(known[i])):
            thecities.append(known[i][j]) # add the country name to the list

            if len(known[i]) >= numOfPulls: # if there is more countries than pulls left, calculate a percentage
                probs.append(round(numOfPulls/len(known[i])*100))
            elif len(known[i]) < numOfPulls: # if not just put 100 because it will certainly happen
                probs.append(100)  

        numOfPulls -= min(numOfPulls,len(known[i])) # subtract the length of the deck or reduce it to zero
    for c in range(len(city)):
        thecities.append(city[c])# add the country name to the list
        if numOfPulls > 0:
            probs.append(round(numOfPulls/len(city)*100))
        else:
            probs.append(0)



    percent_df = pd.DataFrame({"Cities": thecities, "Probability": probs})
    return(percent_df)

In [232]:
## complete end of games mechanisms
#def EndOfGame():
    

## Start by generating game files 

In [54]:
# determine if its a restarted game or starting an original game
gamestatus = input('Are you starting a new game? Yes=1 No=0:  ')

## where the game_logs are stored
logdir = 'game_logs'
citydir = 'city_logs'

if gamestatus == '0':
    ## get relevant file name information
    filedate = input('What date was your game? MMDDYY format:  ')
    gamenum = input('What number game are you playing today?:  ')
    filename = os.path.join(logdir,'POgame_' + filedate + '_' + gamenum + ".csv")#the saved file name
    ####
elif gamestatus == '1':
    ## find out what month this is and what attempt it is:
    gamenum = input('What number game are you playing today?:  ')
    ## get the date
    today = date.today()
    filedate = today.strftime("%m%d%y")
    filename = os.path.join(logdir,'POgame_' + filedate + '_' + gamenum + ".csv")#the saved file name
    build_template(filename)
else:
    print('Oh no! I do not understand that response. Restart please!')
# start game file



Are you starting a new game? Yes=1 No=0:   1
What number game are you playing today?:   5



## load data files and set up relevant information

In [55]:
trial_df = pd.read_csv(filename, delimiter = ',')
trial_df

,Round,PandemicNum,Pandemic,City,Added


In [63]:
## load the trial handler into a pandas structure
trial_df = pd.read_csv(filename, delimiter = ',')

## gather all the countries to be included 
cities_df = pd.read_csv(os.path.join(citydir,'OG_cities.csv'))

with open(os.path.join(citydir,'OG_cities.csv')) as csvfile:
    cities_reader = csv.reader(csvfile, delimiter = '\t')
    cities_x = list(cities_reader)
csvfile.close()


# start by generating the list of cities
city = [x[0] for x in cities_x]
startingPullNum = 6

# catch up if the game had already started 
if trial_df.size:
    revealed, known, city, pandemicNum ,roundNum = catchup(city, trial_df)
else: # assign starting values 
    revealed = []
    known = []
    pandemicNum = 0
    roundNum = 0
    
pandemicflip = [2,2,2,3,3,4,4]

for x in range(20):
    # start with added countries
    gameover = int(input('Game over? Yes[1], No[0]:'))
    
    if gameover:
        trial_df.to_csv(filename, index=False) 
        #EndOfGame
        break
    if roundNum == 0:
        startingPullNum -= trial_df.shape[0]
        for x in range(startingPullNum):
            print(calcprobs(known, city, startingPullNum))
            flippedcity = input('New city ' + str(len(revealed)+ 1) + ': ') # ask what the country is 
            # update revealed and city lists 
            revealed.append(flippedcity)
            city.pop(city.index(flippedcity))
            pandemichuh = 0 # you can't start on a pandemic
            added = 1
            trial_df.loc[len(trial_df)] = [roundNum, pandemicNum,  pandemichuh, flippedcity,  added]
            trial_df.to_csv(filename, index=False) 
    else: 
        print(calcprobs(known, city, pandemicflip[pandemicNum]).head())
        # start by asking if the round is an pandemic 
        pandemichuh = int(input('Pandemic? Yes[1], No[0]:'))
        while pandemichuh:
            flippedcity = input('What is the bottom card?:') # determine the bottom card
            # add to the revealed list, remove from the city list, then move the revealed list to the top of the known list
            known.insert(0,revealed)
            revealed.append(flippedcity) 
            city.pop(city.index(flippedcity))
            
            revealed = []
            pandemicNum += 1 # increment the number of pandemics
            trial_df.loc[len(trial_df)] = [roundNum, pandemicNum,  pandemichuh, flippedcity,  added]
            trial_df.to_csv(filename, index=False) 
            pandemichuh = int(input('Pandemic? Yes[1], No[0]:'))
        # as for each card needed to be flipped this turn
        for cardsRevealed in range(pandemicflip[pandemicNum]):
            flippedcity = input('What city is pulled? '+ str(pandemicflip[pandemicNum]- cardsRevealed) + ' left: ')
            if not known:
                revealed.append(flippedcity)
                city.pop(city.index(flippedcity))
                added = 1
            
            # there are known cards in the known deck pull the the first known list
            elif known:
                revealed.append(flippedcity)
                known[0].pop(known[0].index(flippedcity))
                added = 0
                if not known[0]: # remove the first list if its empty
                    known.pop(0)
            trial_df.loc[len(trial_df)] = [roundNum, pandemicNum,  pandemichuh, flippedcity,  added]
            trial_df.to_csv(filename, index=False) 
    roundNum += 1 # there is more than one row per round so increment appropriately

Game over? Yes[1], No[0]: 0


          Cities  Probability
0        Beijing          100
1  San Francisco            3
2       New York            3
3         London            3
4          Milan            3


Pandemic? Yes[1], No[0]: 0
What city is pulled? 2 left:  Beijing
What city is pulled? 1 left:  Milan
Game over? Yes[1], No[0]: 0


          Cities  Probability
0  San Francisco            6
1       New York            6
2         London            6
3  St Petersburg            6
4          Essen            6


Pandemic? Yes[1], No[0]: Essen


ValueError: invalid literal for int() with base 10: 'Essen'

In [50]:
trial_df

,Round PandemicNum Pandemic City Added


In [26]:
city = [x[0] for x in cities_x]
startingPullNum = 6
PullNum = 2

revealed = [];
known = [];

for index, row in trial_df.iterrows():
    if row.Pandemic == 1: # if its a pandemics add the 'bottom card' and remove the ones from known and put them into the revealed pile
        revealed.append(row.City)
        city.pop(city.index(row.City))
        known.insert(0,revealed)
        revealed = [];

    ## if the known deck and reveal deck are empty pull from the cities list and put them in the revealed deck
    elif not known:
        revealed.append(row.City)
        city.pop(city.index(row.City))

    # there are known cards in the known deck pull the the first known list
    elif known:
        revealed.append(row.City)
        known[0].pop(known[0].index(row.City))
        if not known[0]: # remove the first list if its empty
            known.pop(0)
pandemicNum = trial_df.PandemicNum[len(trial_df)-1]
roundNum = trial_df.Round[len(trial_df)-1]

In [28]:
roundNum

0

In [66]:
from shiny import ui

ui.head_content(
    ui.tags.meta(name="robots", content="noindex")
)


<HTMLDependency "headcontent_7988ade87db04f26f9046617bf29b9d2a742ac8d-0.0">

In [23]:

cities = ['New York','DC','Sao Paulo', 'New York']


switch = [widgets.Button(description=name) for name in cities]

combined = widgets.HBox([items for items in switch])
out =widgets.Output()

def upon_clicked(btn):
    #print(f'You Just clicked {btn.description}.', end='\x1b\r')
    #for n in range(len(cities)):
    #    switch[n].style.button_color = 'gray'
    btn.style.button_color = 'pink'
    return(btn.description)
    

for n in range(len(cities)):
    switch[n].on_click(upon_clicked)

display(combined)


Output()
